In [1]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import numpy as np

tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
def cnn_model_fn(features, labels, mode):
  """Model function for CNN."""
  # Input Layer
  input_layer = tf.reshape(features["x"], [-1, 28, 28, 1])

  # Convolutional Layer #1
  conv1 = tf.layers.conv2d(
      inputs=input_layer,
      filters=32,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu6)   #to try other activation function replcae relu6 by leaky_relu or relu
  # Pooling Layer #1
  pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

  # Convolutional Layer #2 and Pooling Layer #2
  conv2 = tf.layers.conv2d(
      inputs=pool1,
      filters=64,
      kernel_size=[5, 5],
      padding="same",
      activation=tf.nn.relu6) #to try other activation function replcae relu6 by leaky_relu or relu
  pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

  # Dense Layer
  pool2_flat = tf.reshape(pool2, [-1, 7 * 7 * 64])
  dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu6) #to try other activation function replcae relu6 by leaky_relu or relu
  dropout = tf.layers.dropout(
      inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

  # Logits Layer
  logits = tf.layers.dense(inputs=dropout, units=10)

  predictions = {
      # Generate predictions (for PREDICT and EVAL mode)
      "classes": tf.argmax(input=logits, axis=1),
      # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
      # `logging_hook`.
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
  }

  if mode == tf.estimator.ModeKeys.PREDICT:
    return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

  # Calculate Loss (for both TRAIN and EVAL modes)
  loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

  # Configure the Training Op (for TRAIN mode)
  if mode == tf.estimator.ModeKeys.TRAIN:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
    train_op = optimizer.minimize(
        loss=loss,
        global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

  # Add evaluation metrics (for EVAL mode)
  eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(
          labels=labels, predictions=predictions["classes"])
  }
  return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [3]:
# Load training and eval data
((train_data, train_labels),
 (eval_data, eval_labels)) = tf.keras.datasets.mnist.load_data()

train_data = train_data/np.float32(255)
train_labels = train_labels.astype(np.int32)  # not required

eval_data = eval_data/np.float32(255)
eval_labels = eval_labels.astype(np.int32)  # not required

In [4]:

# Create the Estimator# Creat 
mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model_fn)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_service': None, '_is_chief': True, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_save_checkpoints_secs': 600, '_task_type': 'worker', '_save_summary_steps': 100, '_model_dir': '/tmp/tmpoqh1moag', '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_master': '', '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_task_id': 0, '_num_worker_replicas': 1, '_num_ps_replicas': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fd4f54aa5f8>}


In [5]:
# Set up logging for predictions
#tensors_to_log = {"probabilities": "softmax_tensor"}

#logging_hook = tf.train.LoggingTensorHook(
#    tensors=tensors_to_log, every_n_iter=50)

In [6]:
# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
# train one step and display the probabilties
#mnist_classifier.train(input_fn=train_input_fn, steps=1,hooks=[logging_hook])
mnist_classifier.train(input_fn=train_input_fn, steps=1,)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpoqh1moag/model.ckpt.
INFO:tensorflow:loss = 2.3060253, step = 1
INFO:tensorflow:Loss for final step: 2.3060253.


In [7]:
mnist_classifier.train(input_fn=train_input_fn, steps=1000)


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/tmpoqh1moag/model.ckpt-1
INFO:tensorflow:Saving checkpoints for 2 into /tmp/tmpoqh1moag/model.ckpt.
INFO:tensorflow:loss = 2.2937212, step = 2
INFO:tensorflow:global_step/sec: 3.70437
INFO:tensorflow:loss = 2.2799726, step = 102 (26.996 sec)
INFO:tensorflow:global_step/sec: 3.55819
INFO:tensorflow:loss = 2.2519283, step = 202 (28.104 sec)
INFO:tensorflow:global_step/sec: 3.48202
INFO:tensorflow:loss = 2.247101, step = 302 (28.719 sec)
INFO:tensorflow:global_step/sec: 3.45502
INFO:tensorflow:loss = 2.2043853, step = 402 (28.943 sec)
INFO:tensorflow:global_step/sec: 3.28948
INFO:tensorflow:loss = 2.1903968, step = 502 (30.400 sec)
INFO:tensorflow:global_step/sec: 3.16197
INFO:tensorflow:loss = 2.1482084, step = 602 (31.625 sec)
INFO:tensorflow:global_step/sec: 3.37466
INFO:tensorflow:loss = 2.0636399, step = 702 (29.632 sec)
INFO:tensorflow:global_step/sec: 3.26177
INFO:tensorflow:loss = 1.9757806,

In [8]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": eval_data},
    y=eval_labels,
    num_epochs=1,
    shuffle=False)

eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Starting evaluation at 2019-06-27-13:56:18
INFO:tensorflow:Restoring parameters from /tmp/tmpoqh1moag/model.ckpt-1001
INFO:tensorflow:Finished evaluation at 2019-06-27-13:56:29
INFO:tensorflow:Saving dict for global step 1001: accuracy = 0.7155, global_step = 1001, loss = 1.7502644
{'loss': 1.7502644, 'global_step': 1001, 'accuracy': 0.7155}
